In [7]:
from azure.storage.blob import BlobServiceClient
import json
import re
import unicodedata
from math import ceil
from datetime import datetime
import os

In [8]:
data_path = "../local_tests_data/relevant_articles_content/"

articles_filenames = os.listdir(data_path)

n_articles_to_process = len(articles_filenames)

print(f"Number of articles to process: {n_articles_to_process}")

Number of articles to process: 41


In [9]:
prompts_per_batch_job = 200
n_batch_jobs = ceil(n_articles_to_process/prompts_per_batch_job)
print("Creating {} batch files".format(n_batch_jobs))

Creating 1 batch files


In [ ]:
system_prompt = """
You are a professional content assistant summarizing articles for a LinkedIn thought leader. Summarize the article in 3 to 8 sentences. Focus on:

    The main idea or thesis
    The most important insight or data
    Why it matters to professionals or business leaders

    Use clear, neutral, and professional language. Avoid fluff or opinion.

You must summarize the article in English, even if the article is in another language.
"""

In [11]:
def format_task_jsonl_line(task_id, deployment_name, user_input):
    jsonl_line_template = {
        "custom_id": task_id,
        "method": "POST",
        "url": "/chat/completions",
        "body": {
            "model": deployment_name,
            "messages": [
                {
                    "role": "system",
                    "content": system_prompt
                },
                {
                    "role": "user",
                    "content": user_input
                }
            ]
        }
    }
    return jsonl_line_template

In [ ]:
def generate_jsonl_lines(chunk_id, chunk_articles_filenames):
    for j,article_filename in enumerate(chunk_articles_filenames):
        article_id = article_filename.split(".")[0]
        with open(data_path + article_filename, "r") as f:
            article_content = json.load(f)["content"]
        task_id = f"article-summarization-v0*{article_id}"
        deployment_name = "gpt-4o--batch-2"
        yield json.dumps(format_task_jsonl_line(task_id=task_id, deployment_name=deployment_name, user_input=article_content)) + "\n"
    

In [14]:
for i in range(n_batch_jobs):
    print(i)
    chunk = articles_filenames[i*prompts_per_batch_job:min(n_articles_to_process, (i+1)*prompts_per_batch_job)]
    batchfilename = f"article-summarization-v0_{i}_BATCHFILE.jsonl"
    with open(batchfilename, "w") as f:
        for line in generate_jsonl_lines(chunk_id=i, chunk_articles_filenames=chunk):
            f.write(line)

0
